In [3]:
import os
from dotenv import load_dotenv
load_dotenv()
qgenie_api_key = os.getenv("QGENIE_API_KEY")

Using LLMs in LangChain

In [7]:
from qgenie import QGenieClient, ChatMessage
client = QGenieClient(endpoint="https://qgenie-chat.qualcomm.com", api_key=qgenie_api_key )

chat_response = client.chat(
    messages=[ChatMessage(role="user", content="The sky is")],
    max_tokens=400,
)

print(chat_response.first_content)



blue.


Roles example:

System role
Used for instructions the model should use to answer a user question

User role
Used for the user’s query and any other content produced by the user

Assistant role
Used for content generated by the model

In [10]:
from langchain_core.messages import HumanMessage
from qgenie.integrations.langchain import QGenieChat

model = QGenieChat(model="Turbo")
prompt = [HumanMessage("What is the capital of France?")]

model.invoke(prompt)

AIMessage(content='\n\nThe capital of France is Paris.', additional_kwargs={}, response_metadata={'token_usage': {'prompt_tokens': 22, 'total_tokens': 31, 'completion_tokens': 9}, 'model': 'Turbo', 'finish_reason': 'FinishReason.stop'}, id='run--e37cea81-6116-40c3-beee-813db048ea49-0', usage_metadata={'input_tokens': 22, 'output_tokens': 9, 'total_tokens': 31})

Different chat messages

HumanMessage
A message sent from the perspective of the human, with the user role

AIMessage
A message sent from the perspective of the AI that the human is interacting with, with the assistant role

SystemMessage
A message setting the instructions the AI should follow, with the system role

ChatMessage
A message allowing for arbitrary setting of role

In [ ]:
from langchain_core.messages import SystemMessage

system_msg = SystemMessage(
    'You are a helpful assistant that responds to questions with three exclamation marks'
)

human_msg = HumanMessage("what is the capital of France?")

model.invoke([system_msg, human_msg])

AIMessage(content='\n\nThe capital of France is Paris!!!', additional_kwargs={}, response_metadata={'token_usage': {'prompt_tokens': 37, 'total_tokens': 46, 'completion_tokens': 9}, 'model': 'Turbo', 'finish_reason': 'FinishReason.stop'}, id='run--f61d68c8-0361-41c3-a889-d442dfd010af-0', usage_metadata={'input_tokens': 37, 'output_tokens': 9, 'total_tokens': 46})

Making LLM Prompts Reusable using prompt templates

In [15]:
from langchain_core.prompts import PromptTemplate

template = '''
Answer the question based on the context below. If the question cannot be
answered using the information provided, answer with "I don't know".

Context: {context}

Question: {question}

Answer:
 '''

template = PromptTemplate.from_template(template)

prompt = template.invoke({
    "context": '''
        The most recent advancements in NLP are being driven by Large 
        Language Models (LLMs). These models outperform their smaller 
        counterparts and have become invaluable for developers who are creating 
        applications with NLP capabilities. Developers can tap into these 
        models through Hugging Face's `transformers` library, or by utilizing 
        OpenAI and Cohere's offerings through the `openai` and `cohere` 
        libraries, respectively.
    ''',
"question": ''' 
        Which model providers offer LLMs?
    '''
})

completion = model.invoke(prompt)

In [16]:
completion

AIMessage(content='\n\nHugging Face, OpenAI, and Cohere offer LLMs.', additional_kwargs={}, response_metadata={'token_usage': {'prompt_tokens': 158, 'total_tokens': 175, 'completion_tokens': 17}, 'model': 'Turbo', 'finish_reason': 'FinishReason.stop'}, id='run--8e5fad74-eb59-4bf3-b87c-fb249bcfeb0e-0', usage_metadata={'input_tokens': 158, 'output_tokens': 17, 'total_tokens': 175})

Using a role base chat prompt template

In [18]:
from langchain_core.prompts import ChatPromptTemplate

template = ChatPromptTemplate.from_messages([
    ('system', ''' Answer the question based on the context below. If the 
        question cannot be answered using the information provided, answer with 
        "I don\'t know".''' ),
    ('human', 'Context: {context}'),
    ('human', 'Context: {question}'),
])

prompt = template.invoke({
    "context": ''' The most recent advancements in NLP are being driven by Large 
        Language Models (LLMs). These models outperform their smaller 
        counterparts and have become invaluable for developers who are creating 
        applications with NLP capabilities. Developers can tap into these 
        models through Hugging Face's `transformers` library, or by utilizing 
        OpenAI and Cohere's offerings through the `openai` and `cohere` 
        libraries, respectively. ''', 
    "question": "Which model providers offer LLMs?"
})

model.invoke(prompt)

AIMessage(content='\n\nHugging Face, OpenAI, and Cohere offer LLMs.', additional_kwargs={}, response_metadata={'token_usage': {'prompt_tokens': 153, 'total_tokens': 170, 'completion_tokens': 17}, 'model': 'Turbo', 'finish_reason': 'FinishReason.stop'}, id='run--656d987b-ce69-499d-9577-74c6cd7a1082-0', usage_metadata={'input_tokens': 153, 'output_tokens': 17, 'total_tokens': 170})

Getting Specific Formats out of LLMs such as JSON

In [19]:
from langchain_core.pydantic_v1 import BaseModel

class AnswerWithJustificstion(BaseModel):
    '''An answer to the user's question along with justification for the 
        answer.'''
    answer: str
    '''The Answer to the user's question'''
    justification: str
    '''Justification for the answer'''

llm = QGenieChat(model="Pro", max_tokens=512)

structured_llm = llm.with_structured_output(AnswerWithJustificstion)

structured_llm.invoke('''What weighs more, a pound of bricks or a pound 
    of feathers ''')

AnswerWithJustificstion(answer='They weigh the same', justification='A pound is a unit of weight or mass, so one pound of bricks and one pound of feathers both weigh the same amount, one pound.')

Other Machine-Readable Formats with Output Parsers

Output parsers are classes that help you structure large language model responses. They serve two functions:

Providing format instructions
Output parsers can be used to inject some additional instructions in the prompt that will help guide the LLM to output text in the format it knows how to parse.

Validating and parsing output
The main function is to take the textual output of the LLM or chat model and render it to a more structured format, such as a list, XML, or other format. This can include removing extraneous information, correcting incomplete output, and validating the parsed values.

In [22]:
from langchain_core.output_parsers import CommaSeparatedListOutputParser

parser = CommaSeparatedListOutputParser()

items = parser.invoke("apple, banana, cherry")

print(items)

['apple', 'banana', 'cherry']


Assembling the Many Pieces of an LLM Application

Using the Runnable Interface:

- There is a common interface with these methods:

    - invoke: transforms a single input into an output. Takes a single input and returns a single output.
    - batch: efficiently transforms multiple inputs into multiple outputs. takes a list of outputs and returns a list of outputs.
    - stream: streams output from a single input as it’s produced. takes a single input and returns an iterator of parts of the output as they become available.
- There are built-in utilities for retries, fallbacks, schemas, and runtime configurability.
- In Python, each of the three methods have asyncio equivalents.

In [25]:
completion = model.invoke("Hi there!")
print(completion.content)



It's nice to meet you. Is there something I can help you with or would you like to chat?


In [27]:
completions = model.batch(["Hi there!", "Bye!"])
completions

for c in completions:
    print(c.content)



It's nice to meet you. Is there something I can help you with or would you like to chat?


Bye! It was nice chatting with you. If you ever need anything or just want to chat, feel free to come back anytime. Have a great day!


In [ ]:
for token in model.stream("Bye!"):
    print(token.content)




By
e! It
 was nice
 chatting with
 you
. If
 you ever
 need anything or
 just want to
 chat
, feel free to come
 back
 anytime.
 Have a great
 day!



Imperative Composition
Imperative composition is just a fancy name for writing the code you’re used to writing, composing these components into functions and classes. Here’s an example combining prompts, models, and output parsers:

In [ ]:
from langchain_core.runnables import chain

# the building blocks
template = ChatPromptTemplate.from_messages([
    ('system', 'You are a hlepful assistant'),
    ("human", "{question}")
])

# combine them in a function
# @chain decorator adds the same Runnable interface for any function you write

@chain
def chatbot(values):
    prompt = template.invoke(values)
    return model.invoke(prompt)



AIMessage(content="\n\nHere are some of the top car manufacturers that offer high-performing hybrid cars:\n\n1. **Toyota**: Known for their Prius, Toyota offers a range of hybrid models, including the Prius Prime, Camry Hybrid, and RAV4 Hybrid. Their hybrids are known for their fuel efficiency and reliability.\n2. **Honda**: Honda's hybrid lineup includes the Insight, CR-V Hybrid, and Accord Hybrid. Their hybrids are praised for their smooth acceleration and excellent fuel economy.\n3. **Hyundai**: Hyundai's hybrid models, such as the Ioniq Hybrid and Kona Hybrid, offer impressive fuel efficiency and a range of features, including a comfortable ride and advanced safety features.\n4. **Ford**: Ford's hybrid lineup includes the Fusion Hybrid, Escape Hybrid, and Explorer Hybrid. Their hybrids are known for their performance and innovative features, such as the ability to switch between electric and gasoline power.\n5. **Lexus**: As a luxury brand, Lexus offers high-performance hybrids lik

In [32]:
# use it
print(chatbot.invoke({"question": "What car manufacturers see the best performing hybrid cars?"}).content)



Here are some of the top car manufacturers that offer high-performing hybrid cars:

1. **Toyota**: Known for their Prius, Toyota offers a range of hybrid models, including the Prius Prime, Camry Hybrid, and RAV4 Hybrid. Their hybrids are known for their fuel efficiency and reliability.
2. **Honda**: Honda's hybrid lineup includes the Insight, CR-V Hybrid, and Accord Hybrid. Their hybrids are praised for their smooth acceleration and excellent fuel economy.
3. **Hyundai**: Hyundai's hybrid models, such as the Ioniq Hybrid and Kona Hybrid, offer impressive fuel efficiency and a range of features like adaptive cruise control and lane departure warning.
4. **Ford**: Ford's hybrid lineup includes the Fusion Hybrid, Escape Hybrid, and Explorer Hybrid. Their hybrids are known for their sporty performance and advanced technology features.
5. **Lexus**: As a luxury brand, Lexus offers high-performance hybrids like the ES Hybrid, GS Hybrid, and RX Hybrid. Their hybrids are known for their smoo

A streamable version of the above

In [34]:
@chain
def chatbot(values):
    prompt = template.invoke(values)
    for token in model.stream(prompt):
        yield token

for part in chatbot.stream({
    "question": "Which model providers offer LLMs?"
}):
    print(part.content)




Several
 model
 providers offer Large Language
 Models (LLMs
). Here are some of the
 most notable
 ones:

1. **H
ugging Face
 Transformers
**: Hugging Face offers
 a wide
 range of pre
-trained L
LMs, including B
ERT
, RoBERTa
, DistilBERT
, and many
 others
,
 through their Transformers
 library.
2. **Google
 Cloud
 AI
 Platform
**: Google Cloud offers
 a range
 of L
LMs, including B
ERT, Ro
BERTa, and AL
BERT
, through their AI
 Platform.
3. **Microsoft
 Azure Cognitive
 Services
**: Microsoft Azure offers
 a range of LLMs
, including B
ERT, RoBERTa,
 and Turing
-N
LG
, through their Cognitive
 Services.
4. **IBM
 Watson Studio
**: IBM Watson Studio
 offers a range of LLM
s, including B
ERT, Ro
BERTa, and U
LM
Fi
T
, through their Watson
 Studio platform
.
5. **Amazon Sage
Maker
**: Amazon SageMaker offers a
 range of LLMs,
 including BERT, RoBERT
a, and Dist
ilBERT, through their Sage
Maker platform.
6. **
Deep
Mind
**: DeepMind,
 a subsidiary
 of Alphabet
 Inc
.,
 offers a rang

Async verison

In [35]:
@chain
async def chatbot(values):
    prompt = await template.ainvoke(values)
    return await model.ainvoke(prompt)

await chatbot.ainvoke({"question": "Which model providers offer LLMs?"})


AIMessage(content="\n\nSeveral model providers offer Large Language Models (LLMs). Here are some of the most notable ones:\n\n1. **Hugging Face Transformers**: Hugging Face offers a wide range of pre-trained LLMs, including BERT, RoBERTa, DistilBERT, and many others, through their Transformers library.\n2. **Google Cloud AI Platform**: Google Cloud offers a range of LLMs, including BERT, RoBERTa, and XLNet, through their AI Platform.\n3. **Microsoft Azure Cognitive Services**: Microsoft Azure offers a range of LLMs, including BERT, RoBERTa, and Turing-NLG, through their Cognitive Services.\n4. **Amazon SageMaker**: Amazon SageMaker offers a range of LLMs, including BERT, RoBERTa, and XLNet, through their SageMaker platform.\n5. **IBM Watson**: IBM Watson offers a range of LLMs, including BERT, RoBERTa, and ULMFiT, through their Watson platform.\n6. **DeepMind**: DeepMind, a subsidiary of Alphabet Inc., offers a range of LLMs, including AlphaFold and AlphaGo, through their DeepMind plat

Declarative Composition

LCEL is a declarative language for composing LangChain components. LangChain compiles LCEL compositions to an optimized execution plan, with automatic parallelization, streaming, tracing, and async support.

Let’s see the same example using LCEL:

In [36]:
# the building blocks

template = ChatPromptTemplate.from_messages([
    ('system', 'You are a helpful assistant.'),
    ('human', '{question}'),
])


# combine them with the | operator

chatbot = template | model

# use it

chatbot.invoke({"question": "Which model providers offer LLMs?"})

AIMessage(content='\n\nSeveral model providers offer Large Language Models (LLMs). Here are some of the most well-known ones:\n\n1. **Hugging Face Transformers**: Hugging Face offers a wide range of pre-trained LLMs, including BERT, RoBERTa, DistilBERT, and many others, through their Transformers library.\n2. **Google Cloud AI Platform**: Google Cloud offers a range of LLMs, including BERT, RoBERTa, and ALBERT, through their AI Platform.\n3. **Microsoft Azure Cognitive Services**: Microsoft Azure offers a range of LLMs, including BERT, RoBERTa, and Turing-NLG, through their Cognitive Services.\n4. **IBM Watson**: IBM Watson offers a range of LLMs, including BERT, RoBERTa, and ULMFiT, through their Watson Assistant.\n5. **Amazon SageMaker**: Amazon SageMaker offers a range of LLMs, including BERT, RoBERTa, and XLNet, through their SageMaker platform.\n6. **DeepMind**: DeepMind, a subsidiary of Alphabet Inc., offers a range of LLMs, including AlphaBERT and AlphaFold, through their DeepMi